In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import Conv1D, Dropout, SpatialDropout1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk
import re

In [4]:
df = pd.read_csv('/content/gdrive/My Drive/twitter30k.csv')

In [8]:
X = df['twitts'].tolist()
y = df['sentiment']

In [21]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [24]:
x_train = []
for sent in X:
    sent = re.sub("[^a-zA-Z]", " ", sent)
    sent = sent.lower().split()
    sent = [lemmatizer.lemmatize(word) for word in sent if word not in set(stop_words)]
    sent = " ".join(sent)
    x_train.append(sent)

In [31]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [61]:
vocab_size = len(tokenizer.word_index) + 1 

In [42]:
encoded_texts = tokenizer.texts_to_sequences(x_train)

In [46]:
testing_list = []
for l in x_train:
    testing_list.append(len(l.split(' ')))
max_sentence_length = max(testing_list)

In [49]:
padded_X = pad_sequences(encoded_texts, maxlen=max_sentence_length, padding='post', truncating='post')

In [53]:
X_train, X_test, y_train, y_test = train_test_split(padded_X, y, test_size=0.15, random_state=42)

In [58]:
embedding_features = 300

In [87]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_features, input_length=max_sentence_length))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(64, recurrent_activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, recurrent_activation='relu', recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [88]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [89]:
hist = model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
797/797 [==============================] - 313s 392ms/step - loss: 0.6261 - accuracy: 0.6263 - val_loss: 0.5519 - val_accuracy: 0.7187
Epoch 2/10
797/797 [==============================] - 314s 394ms/step - loss: 0.4980 - accuracy: 0.7666 - val_loss: 0.5499 - val_accuracy: 0.7240
Epoch 3/10
797/797 [==============================] - 312s 391ms/step - loss: 0.4248 - accuracy: 0.8144 - val_loss: 0.5823 - val_accuracy: 0.7169
Epoch 4/10
797/797 [==============================] - 310s 389ms/step - loss: 0.3917 - accuracy: 0.8418 - val_loss: 0.6107 - val_accuracy: 0.7156
Epoch 5/10
797/797 [==============================] - 314s 393ms/step - loss: 0.3678 - accuracy: 0.8517 - val_loss: 0.6195 - val_accuracy: 0.7180
Epoch 6/10
797/797 [==============================] - 311s 391ms/step - loss: 0.3474 - accuracy: 0.8647 - val_loss: 0.6436 - val_accuracy: 0.7142
Epoch 7/10
797/797 [==============================] - 312s 392ms/step - loss: 0.3388 - accuracy: 0.8684 - val_loss: 0.6410 -

In [90]:
model.save('/content/gdrive/My Drive/twitter.h5')

In [ ]:
#Testing
from tensorflow.keras.models import load_model

lstm_model = load_model('/content/gdrive/My Drive/twitter.h5')

In [135]:
sentiments = ['negative', 'positive']

In [92]:
def text_preprocess(text):
    encoded = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(encoded, maxlen=max_sentence_length, padding='post', truncating='post')
    return padded

In [152]:
tweet1 = ['i want to kill myself'] #neg sentiment 0
tweet2 = ['thank you very much'] #pos sentiment 1
txt1 = text_preprocess(tweet1)
txt2 = text_preprocess(tweet2)
output1 = sentiments[lstm_model.predict_classes(txt1)[0][0]]
output2 = sentiments[lstm_model.predict_classes(txt2)[0][0]]

In [153]:
print(output1, output2)

negative positive
